## Proceso 4: Clasificación de sentimientos múltiples, como: alegría, ira, culpa, etc.

In [1]:
import re 
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import time

import sys
sys.path.append(sys.path[0] + '/..')

from cajaBlanca import auditoria

In [2]:
def read_data(file):
    data = []
    with open(file, 'r')as f:
        for line in f:
            line = line.strip()
            label = ' '.join(line[1:line.find("]")].strip().split())
            text = line[line.find("]")+1:].strip()
            data.append([label, text])
    return data

In [3]:
file = 'sentimientoMultiple_es.txt'
data = read_data(file)
print("Number of instances: {}".format(len(data)))

Number of instances: 7480


### tokenización y la generación de las características de una oración 

In [4]:
def ngram(token, n): 
    output = []
    for i in range(n-1, len(token)): 
        ngram = ' '.join(token[i-n+1:i+1])
        output.append(ngram) 
    return output

def create_feature(text, nrange=(1, 1)):
    text_features = [] 
    text = text.lower() 
    text_alphanum = re.sub('[^a-z0-9#]', ' ', text)
    for n in range(nrange[0], nrange[1]+1): 
        text_features += ngram(text_alphanum.split(), n)    
    text_punc = re.sub('[a-z0-9]', ' ', text)
    text_features += ngram(text_punc.split(), 1)
    return Counter(text_features)

### Función para almacenar las etiquetas, que se basarán en emociones como Alegría, Miedo, Ira, etc.:

In [5]:
def convert_label(item, name): 
    #print('item: ', item)
    #print('name: ', name)
    items = list(map(float, item.split()))
    label = ""
    for idx in range(len(items)): 
        if items[idx] == 1: 
            label += name[idx] + " "
    
    return label.strip()

emotions = ["alegría", 'miedo', "ira", "tristeza", "disgusto", "vergüenza", "culpa"]

X_all = []
y_all = []
for label, text in data:
    y_all.append(convert_label(label, emotions))
    X_all.append(create_feature(text, nrange=(1, 4)))

### Separación de datos en conjuntos de entrenamiento y prueba

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size = 0.2, random_state = 123)

def train_test(clf, X_train, X_test, y_train, y_test):
    clf.fit(X_train, y_train)
    train_acc = accuracy_score(y_train, clf.predict(X_train))
    test_acc = accuracy_score(y_test, clf.predict(X_test))
    return train_acc, test_acc

from sklearn.feature_extraction import DictVectorizer
vectorizer = DictVectorizer(sparse = True)
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

### Entrenamiento con cuatro modelos de aprendizaje automático para la clasificación múltiple de sentimientos:
1. Clasificación con vectores de Soporte 
2. Clasificación con vectores de soporte lineal
3. Clasificación con randon forest
4. Clasificación con árboles de decisión

In [7]:
svc = SVC()
lsvc = LinearSVC(random_state=100)
rforest = RandomForestClassifier(random_state=100)
dtree = DecisionTreeClassifier()

clifs = [svc, lsvc, rforest, dtree]

evaluacion = {}
modelos = {}
for clf in clifs: 
    tiempo_inicio = time.time()
    clf_name = clf.__class__.__name__
    train_acc, test_acc = train_test(clf, X_train, X_test, y_train, y_test)
    evaluacion['Training Accuracy'] = train_acc
    evaluacion['Test Accuracy'] = test_acc
    modelos[clf_name] = [clf.get_params(), evaluacion]
    auditoria.registrarPistaSentimientosMultiples([tiempo_inicio, "", "", clf_name, modelos[clf_name]])


tiempo inicio:  1664038253.4389513
tiempo final:  75.76993370056152


c:\tools\ides\anaconda\envs\entorno_prueba\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


tiempo inicio:  1664038329.2692733
tiempo final:  11.6302969455719
tiempo inicio:  1664038340.906551
tiempo final:  78.42996120452881
tiempo inicio:  1664038419.3549953
tiempo final:  19.64985752105713


## Resultado de los modelos

In [8]:
for modelo in modelos:
    print(modelo, modelos[modelo][1])

SVC {'Training Accuracy': 0.9988302139037433, 'Test Accuracy': 0.4177807486631016}
LinearSVC {'Training Accuracy': 0.9988302139037433, 'Test Accuracy': 0.4177807486631016}
RandomForestClassifier {'Training Accuracy': 0.9988302139037433, 'Test Accuracy': 0.4177807486631016}
DecisionTreeClassifier {'Training Accuracy': 0.9988302139037433, 'Test Accuracy': 0.4177807486631016}


## Definición de la etiqueta múltiple de sentimientos con su respectivo emoji

In [9]:
l = ["alegría", 'miedo', "ira", "tristeza", "disgusto", "vergüenza", "culpa"]
l.sort()
label_freq = {}
for label, _ in data: 
    label_freq[label] = label_freq.get(label, 0) + 1

# print the labels and their counts in sorted order 
for l in sorted(label_freq, key=label_freq.get, reverse=True):
    print("{:10}({})  {}".format(convert_label(l, emotions), l, label_freq[l]))

emoji_dict = {"alegría":"😂", "miedo":"😱", "ira":"😠", "tristeza":"😢", "disgusto":"😒", "vergüenza":"😳", "culpa":"😳"}

alegría   (1. 0. 0. 0. 0. 0. 0.)  1084
ira       (0. 0. 1. 0. 0. 0. 0.)  1080
tristeza  (0. 0. 0. 1. 0. 0. 0.)  1079
miedo     (0. 1. 0. 0. 0. 0. 0.)  1078
disgusto  (0. 0. 0. 0. 1. 0. 0.)  1057
culpa     (0. 0. 0. 0. 0. 0. 1.)  1057
vergüenza (0. 0. 0. 0. 0. 1. 0.)  1045


## Obtiene letra de canción preprocesada

In [10]:
from bdd import mongoDB_cliente

# Maluma                HAWÁI
# EL BOBE / ALPEDUE     ELLA
# JC LA NEVULA / ATOMIC OTRO WAY        BUSCATE OTRO
cantante = 'JC LA NEVULA / ATOMIC OTRO WAY'
cancion = 'BUSCATE OTRO'
canciones = mongoDB_cliente.obtenerColeccion('CANCIONES', {'cantante': cantante, 'cancion': cancion})   
letra_preprocesada = canciones[0]['letra_preprocesada']
letra_cancion_pre = ''
for letra in letra_preprocesada:
    letra_cancion_pre = letra_cancion_pre + letra + " "
print(letra_cancion_pre)

dije sueltes quiero saber saber viví culpa toca vivirlo vivirlo importa tires voy bloquear whatsapp vacana dijiste voy hacer ver na' búscate soporte celos estupidez sentía murió dije ibas pagar crece ahora llamas contesto quiero peleas mensajes texto déjame tirarme buscando pretexto nueva vida moyeto contigo perdí tiempo reloj caiga mar fatal apreta'o vuelvo dar imagine iba' desafinar tal vez mire rió si miro mal shorty vete casa rompió pozuelo conmigo esta' tan caliente sol bloque hielo aprendí tener palabra dinero na' entonces vemos luego digo pues vemos luego digo pues vemos luego digo vemos luego digo farisea vemos luego búscate soporte celos estupidez sentía murió dije ibas pagar crece ahora llamas contesto quiero peleas mensajes texto déjame tirarme buscando pretexto nueva vida moyeto decía dolía importo sentía puedes rodar quiero vida ahora toca prender fuego vía progreso hooka par cervezas chapeo cabeza quisiste bajarle veo ahora pesa burlado par mami fresa amor mierda metí pam

## Ejecución del modelo con vectores de soporte

In [11]:
tiempo_inicio = time.time()

features = create_feature(letra_cancion_pre, nrange=(1, 4))
features = vectorizer.transform(features)
prediction = svc.predict(features)[0]

resultado = {}
resultado['prediccion'] = prediction
modelos['SVC'].append(resultado)
print('Clasificación de sentimientos con vectores de soporte: ', emoji_dict[prediction], modelos['SVC'][2])
auditoria.registrarPistaSentimientosMultiples([tiempo_inicio, cancion, cantante, 'SVC', modelos['SVC']])


Clasificación de sentimientos con vectores de soporte:  😠 {'prediccion': 'ira'}
tiempo inicio:  1664038439.171931
tiempo final:  0.021956443786621094


## Ejecución del modelo vectores de soporte lineal

In [15]:
tiempo_inicio = time.time()

features = create_feature(letra_cancion_pre, nrange=(1, 4))
features = vectorizer.transform(features)
prediction = lsvc.predict(features)[0]

resultado = {}
resultado['prediccion'] = prediction
modelos['LinearSVC'].append(resultado)
print('Clasificación de sentimientos con vectores de soporte lineal (LinearSVC): ', emoji_dict[prediction], modelos['LinearSVC'][2])
auditoria.registrarPistaSentimientosMultiples([tiempo_inicio, cancion, cantante, 'LinearSVC', modelos['LinearSVC']])

Clasificación de sentimientos con vectores de soporte lineal (LinearSVC):  😢 {'prediccion': 'tristeza'}
tiempo inicio:  1664039095.0251489
tiempo final:  0.005983829498291016


## Ejecución del modelo randon forest

In [16]:
tiempo_inicio = time.time()

features = create_feature(letra_cancion_pre, nrange=(1, 4))
features = vectorizer.transform(features)
prediction = rforest.predict(features)[0]
print(': ', emoji_dict[prediction], prediction)

resultado = {}
resultado['prediccion'] = prediction
modelos['RandomForestClassifier'].append(resultado)
print('Clasificación de sentimientos con randon forest (RandomForestClassifier): ', emoji_dict[prediction], modelos['RandomForestClassifier'][2])
auditoria.registrarPistaSentimientosMultiples([tiempo_inicio, cancion, cantante, 'RandomForestClassifier', modelos['RandomForestClassifier']])

:  😳 culpa
Clasificación de sentimientos con randon forest (RandomForestClassifier):  😳 {'prediccion': 'culpa'}
tiempo inicio:  1664039215.6402233
tiempo final:  0.09977126121520996


# Ejecución del modelo con árboles de decisión

In [17]:
tiempo_inicio = time.time()

features = create_feature(letra_cancion_pre, nrange=(1, 4))
features = vectorizer.transform(features)
prediction = dtree.predict(features)[0]

resultado = {}
resultado['prediccion'] = prediction
modelos['DecisionTreeClassifier'].append(resultado)
print('Clasificación de sentimientos con árboles de decisión (DecisionTreeClassifier): ', emoji_dict[prediction], modelos['DecisionTreeClassifier'][2])
auditoria.registrarPistaSentimientosMultiples([tiempo_inicio, cancion, cantante, 'DecisionTreeClassifier', modelos['DecisionTreeClassifier']])

Clasificación de sentimientos con árboles de decisión (DecisionTreeClassifier):  😢 {'prediccion': 'tristeza'}
tiempo inicio:  1664039301.3923256
tiempo final:  0.003988742828369141
